# Multivariate polynomials implementations

**Contributed by**: Benoît Legat

The SumOfSquares package is built on top of the [MultivariatePolynomials](https://github.com/JuliaAlgebra/MultivariatePolynomials.jl)
abstract interface. [DynamicPolynomials](https://github.com/JuliaAlgebra/DynamicPolynomials.jl/)
is an implementation of this abstract interface so it can be used with
SumOfSquares. Moreover, any other implementation can be used as well. To
illustrate, we solve of [Blekherman2012; Examples 3.38](@cite) with
[TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl),
another implementation of [MultivariatePolynomials](https://github.com/JuliaAlgebra/MultivariatePolynomials.jl).

In [1]:
import TypedPolynomials
TypedPolynomials.@polyvar x y
using SumOfSquares
import CSDP
model = SOSModel(CSDP.Optimizer)
con_ref = @constraint(model, 2x^4 + 5y^4 - x^2*y^2 >= -2(x^3*y + x + 1))
optimize!(model)
solution_summary(model)

CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.39e-01 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  9.2771541e+01 
Iter:  2 Ap: 1.00e+00 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  5.3405053e+01 


* Solver : CSDP

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 4.30107e-04


We see that the problem is feasible. The Sum-of-Squares decomposition can be
obtained as follows:

In [2]:
sos_decomposition(con_ref)

(-0.6797543969221541·1 - 0.09754507709792246·y - 0.4175950563713448·x + 2.1931518901396174·y^2 - 0.1269606244678339·x*y - 0.8025867547845221·x^2)^2 + (0.9043573827480671·1 - 0.6455003192597278·y + 0.2742159536999832·x - 0.10770174782358159·y^2 - 1.2293138068800717·x*y - 0.9300160939847·x^2)^2 + (-0.5739168240154698·1 - 1.38535050221548·y - 0.6219655378569912·x - 0.2040924100740057·y^2 - 0.15182215880489783·x*y + 0.4443832632766482·x^2)^2 + (0.5703021360913615·1 - 0.5184093413621602·y + 0.3436728787624473·x + 0.2555950605470665·y^2 + 0.7618441294650047·x*y - 0.02090567376761348·x^2)^2 + (0.041400837217767876·1 + 0.04911040272637946·y - 0.49990658891857337·x - 0.24792145939081137·y^2 + 0.2972445689917418·x*y - 0.5054184235227418·x^2)^2 + (-0.2524531788141328·1 - 0.06336647012459033·y + 0.253942572224051·x - 0.10018744153048117·y^2 + 0.059601795985895156·x*y - 0.19381241255811912·x^2)^2

Why is there several implementations ?
Depending in the use-case, one implementation may be more appropriate than
another one. [TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl)
is faster than [DynamicPolynomials](https://github.com/JuliaAlgebra/DynamicPolynomials.jl/)
but it requires new compilation whenever the list of variables changes.
This means that [TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl)
is not appropriate when the number of variables is dynamic or too large.
However, for a small number of variables, it can be faster.
When solving Sum-of-Squares programs, the time is mostly taken by the Semidefinite programming solver.
The time taken by SumOfSquares/JuMP/MathOptInterface are usually negligible
or it time is taken by manipulation of JuMP or MathOptInterface functions
therefore using TypedPolynomials over DynamicPolynomials may not make much difference in most cases.

One case for which using TypedPolynomials might be adequate is when
using domain defined by equalities (possibly also with inequalities).
Indeed, in that case, SumOfSquares computes the corresponding Gröbner basis which
may take a non-negligible amount of time for large systems of equalities.

To illustrate this, consider the computation of Gröbner basis for the
following system from [CLO05, p. 17].
The time taken by TypedPolynomials is below:

[CLO05] Cox, A. David & Little, John & O'Shea, Donal
*Using Algebraic Geometry*.
Graduate Texts in Mathematics, **2005**.
https://doi.org/10.1007/b138611

In [3]:
using BenchmarkTools
@btime let
    TypedPolynomials.@polyvar x y
    S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
    SemialgebraicSets.compute_gröbner_basis!(S.I)
end

  197.801 μs (5860 allocations: 1.30 MiB)


true

The time taken by DynamicPolynomials is as follows:

In [4]:
import DynamicPolynomials
@btime let
    DynamicPolynomials.@polyvar x y
    S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
    SemialgebraicSets.compute_gröbner_basis!(S.I)
end

  382.286 μs (10893 allocations: 1.22 MiB)


true

We see that TypedPolynomials is faster.
The time is still negligible for this small system but for larger systems, choosing TypedPolynomials may be helpful.
We can use this system in a Sum-of-Squares constraint as follows:

In [5]:
TypedPolynomials.@polyvar x y
S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
poly = -6x - 4y^3 + 2x*y^2 + 6x^3 - 3y^4 + 13x^2 * y^2
model = Model(CSDP.Optimizer)
con_ref = @constraint(model, poly in SOSCone(), domain = S)
optimize!(model)
solution_summary(model)

Success: SDP solved
Primal objective value: 0.0000000e+00 
Dual objective value: 0.0000000e+00 
Relative primal infeasibility: 2.23e-16 
Relative dual infeasibility: 5.00e-11 
Real Relative Gap: 0.00e+00 
XZ Relative Gap: 3.04e-10 
DIMACS error measures: 2.78e-16 0.00e+00 5.00e-11 0.00e+00 0.00e+00 3.04e-10
CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.64e-01 Pobj:  0.0000000e+00 Ad: 8.07e-01 Dobj:  2.6104524e+00 
Iter:  2 Ap: 7.71e-01 Pobj:  0.0000000e+00 Ad: 7.40e-01 Dobj: -5.9092654e-02 
Iter:  3 Ap: 8.08e-01 Pobj:  0.0000000e+00 Ad: 7.65e-01 Dobj:  5.5549062e-01 
Iter:  4 Ap: 7.56e-01 Pobj:  0.0000000e+00 Ad: 7.79e-01 Dobj:  4.8506833e-01 
Iter:  5 Ap: 6.95e-01 Pobj:  0.0000000e+00 Ad: 6.77e-01 Dobj:  4.3029691e-01 
Iter:  6 Ap: 6.35e-01 Pobj:  0.0000000e+00 Ad: 6.83e-01 Dobj:  2.2137596e-01 
Iter:  7 Ap: 6.21e-01 Pobj:  0.0000000e+00 Ad: 6.43e-01 Dobj:  1.2986380e-01 
Iter:  8 Ap: 

* Solver : CSDP

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 2.37894e-03


We obtain the following decomposition:

In [6]:
dec = sos_decomposition(con_ref, 1e-6)

(-6.841219256090771e-5·1 + 0.0006527330340649107·y - 0.00014150321438476302·x + 7.044882232835532·y^2 - 28.229546532117844·x*y + 25.36354533708025·x^2)^2 + (-1.4193710561626973e-5·1 + 0.00042746788877302523·y + 0.0008136453704606653·x + 22.479251122164996·y^2 - 13.864839113487497·x*y - 21.675274123197855·x^2)^2 + (-3.84390450630064e-5·1 - 0.0007285126695822746·y + 0.0005524428333021625·x + 0.12018797686157888·y^2 + 0.09016527012493944·x*y + 0.06697072803793322·x^2)^2 + (-6.125336122569302e-7·1 - 0.03996370218393435·y + 0.05340324712849221·x - 0.0002606951920930786·y^2 - 0.00019563679757630243·x*y - 0.00014400703006102608·x^2)^2

We can verify that it is correct as follows:

In [7]:
rem(dec - poly, S.I)

6.35962489327993912288384867185532522970703439568751491606235504150390625e-09 - 2.031836040539138034770952477781396624733608801668593421196326231345130675312378e-07y - 1.651283078137803036392702219577936025766225961538461538461538461538458123853356e-07x - 1.3042138486059584625274300473396493771360837854444980621337890625e-05y² - 1.9531084959490221421296585191385020152665674686431884765625e-05xy - 7.3007271062877905661698228756417705653802840970456600189208984375e-06x² - 4.626728363395689787770000596841176350911458333333333333333333333333335636311615e-08y³ - 4.979310060567361991464470823605855305989583333333333333333333333333332181844193e-08xy² - 4.16004339042067556686359253159679383315960876643657684326171875e-08x²y + 1.428514469108239819224064166729266826923076923076923076923076923076649908268473e-08y⁴

Note that the difference between `dec` and `poly` is larger
than between the full gram matrix because `dec` is obtained by dropping
the lowest eigenvalues with the threshold `1e-6`; see `sos_decomposition`.

In [8]:
rem(gram_matrix(con_ref) - poly, S.I)

7.413986070507335123927063571559747945372009780840016901493072509765625e-09 + 1.320663966652354727218799883014649792628985708758521538514357346754875984471248e-08y - 3.175694510778507370000275281759408804086538461538461538461538461538120077643758e-09x + 7.41404251511888767733893246258958242833614349365234375e-09y² - 4.2730576021998700753101729787886142730712890625e-14xy + 7.413996962364610698870137639460153877735137939453125e-09x² - 3.4816594052244909107685089111328125e-13y³ + 6.85229650798646616749465465545654296875e-13xy² + 3.443079155118766720988787710666656494140625e-14x²y + 1.434300254634133755014492915226862980769230769230769230769230769230496062114595e-08y⁴

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*